<a href="https://colab.research.google.com/github/as791/stAdv-PyTorch/blob/main/StAdv_attack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms,datasets
from scipy import optimize
import random

from tqdm.notebook import tqdm
from PIL import Image

##Train Model

In [ ]:
class NetA(nn.Module):
    def __init__(self):
      super().__init__()
      self.model = nn.Sequential(
        nn.Conv2d(1, 64, kernel_size=5),
        nn.ReLU(),
        nn.Conv2d(64, 64, kernel_size=5),
        nn.ReLU(),
        nn.Flatten(),
        nn.Dropout(0.25),
        nn.Linear(64*20*20, 128),
        nn.Dropout(0.5),
        nn.Linear(128, 10),
      )

    def forward(self, x):
      return self.model(x)


class NetB(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(
      nn.Conv2d(1, 64, kernel_size=8),
      nn.ReLU(),
      nn.Dropout(0.2),
      nn.Conv2d(64, 128, kernel_size=6),
      nn.ReLU(),
      nn.Conv2d(128, 128, kernel_size=5),
      nn.ReLU(),
      nn.Flatten(),
      nn.Dropout(0.5),
      nn.Linear(128*12*12, 10)
    )

  def forward(self, x):
    return self.model(x)


class NetC(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(
      nn.Conv2d(1, 128, 3),
      nn.ReLU(),
      nn.Conv2d(128, 64, 3),
      nn.ReLU(),
      nn.Flatten(),
      nn.Dropout(0.25),
      nn.Linear(64*24*24, 128),
      nn.ReLU(),
      nn.Dropout(0.5),
      nn.Linear(128, 10)
    )

  def forward(self, x):
    return self.model(x)

In [ ]:
def fit(model,device,train_loader,val_loader,optimizer,criterion,epochs):
  data_loader = {'train':train_loader,'val':val_loader}
  print("Fitting the model...")
  train_loss,val_loss=[],[]
  train_acc,val_acc=[],[]

  for epoch in tqdm(range(epochs)):
    loss_per_epoch,val_loss_per_epoch=0,0
    acc_per_epoch,val_acc_per_epoch,total,val_total=0,0,0,0
    for phase in ('train','val'):
      for i,data in tqdm(enumerate(data_loader[phase]), total= (430 if phase == "train" else 40)):
        inputs,labels  = data[0].to(device),data[1].to(device)
        outputs = model(inputs)
        #preding classes of one batch
        preds = torch.max(outputs,1)[1]
        #calculating loss on the output of one batch
        loss = criterion(outputs,labels)
        if phase == 'train':
          acc_per_epoch+=(labels==preds).sum().item()
          total+= labels.size(0)
          optimizer.zero_grad()
          #grad calc w.r.t Loss func
          loss.backward()
          #update weights
          optimizer.step()
          loss_per_epoch+=loss.item()
        else:
          val_acc_per_epoch+=(labels==preds).sum().item()
          val_total+=labels.size(0)
          val_loss_per_epoch+=loss.item()
    print("Epoch: {} Loss: {:0.6f} Acc: {:0.6f} Val_Loss: {:0.6f} Val_Acc: {:0.6f}".format(epoch+1,loss_per_epoch/len(train_loader),acc_per_epoch/total,val_loss_per_epoch/len(val_loader),val_acc_per_epoch/val_total))
    train_loss.append(loss_per_epoch/len(train_loader))
    val_loss.append(val_loss_per_epoch/len(val_loader))
    train_acc.append(acc_per_epoch/total)
    val_acc.append(val_acc_per_epoch/val_total)
  return train_loss,val_loss,train_acc,val_acc

In [ ]:
models = [NetA(), NetB(), NetC()]

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
dataset = datasets.MNIST(root = 'data', train=True, transform = transform, download=True)
train_set, val_set = torch.utils.data.random_split(dataset, [55000, 5000])
train_loader = torch.utils.data.DataLoader(train_set,batch_size=128,shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set,batch_size=128,shuffle=True)
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# device = torch.device("cuda")
print(device)
for model in models:
  model = model.to(device)
  optimizer = optim.SGD(model.parameters(),lr=0.01, momentum=0.9, nesterov=True, weight_decay=1e-6)
  train_loss,val_loss,train_acc,val_acc=fit(model,device,train_loader,val_loader,optimizer,criterion,20)
  torch.save(model.state_dict(),f'./{type(model).__name__}.pt')


In [ ]:
fig = plt.figure(figsize=(5,5))
plt.plot(np.arange(1,21), train_loss, "*-",label="Training Loss")
plt.plot(np.arange(1,21), val_loss,"o-",label="Val Loss")
plt.xlabel("Num of epochs")
plt.legend()
plt.show()
# plt.savefig('loss_event.png')

fig = plt.figure(figsize=(5,5))
plt.plot(np.arange(1,21), train_acc, "*-",label="Training Acc")
plt.plot(np.arange(1,21), val_acc,"o-",label="Val Acc")
plt.xlabel("Num of epochs")
plt.legend()
plt.show()
# plt.savefig('accuracy_event.png')

##stAdv Attack

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#https://github.com/rakutentech/stAdv - refered code
def flow_st(images,flows):
  images_shape = images.size()
  flows_shape = flows.size()
  batch_size = images_shape[0]
  H = images_shape[2]
  W = images_shape[3]
  basegrid = torch.stack(torch.meshgrid(torch.arange(0,H), torch.arange(0,W))) #(2,H,W)
  sampling_grid = basegrid.unsqueeze(0).type(torch.float32).to(device) + flows.to(device)
  sampling_grid_x = torch.clamp(sampling_grid[:,1],0.0,W-1.0).type(torch.float32)
  sampling_grid_y = torch.clamp(sampling_grid[:,0],0.0,H-1.0).type(torch.float32)

  x0 = torch.floor(sampling_grid_x).type(torch.int64)
  x1 = x0 + 1
  y0 = torch.floor(sampling_grid_y).type(torch.int64)
  y1 = y0 + 1

  x0 = torch.clamp(x0, 0, W - 2)
  x1 = torch.clamp(x1, 0, W - 1)
  y0 = torch.clamp(y0, 0, H - 2)
  y1 = torch.clamp(y1, 0, H - 1)

  Ia = images[:,:,y0[0,:,:], x0[0,:,:]]
  Ib = images[:,:,y1[0,:,:], x0[0,:,:]]
  Ic = images[:,:,y0[0,:,:], x1[0,:,:]]
  Id = images[:,:,y1[0,:,:], x1[0,:,:]]

  x0 = x0.type(torch.float32)
  x1 = x1.type(torch.float32)
  y0 = y0.type(torch.float32)
  y1 = y1.type(torch.float32)

  wa = (x1 - sampling_grid_x) * (y1 - sampling_grid_y)
  wb = (x1 - sampling_grid_x) * (sampling_grid_y - y0)
  wc = (sampling_grid_x - x0) * (y1 - sampling_grid_y)
  wd = (sampling_grid_x - x0) * (sampling_grid_y - y0)

  perturbed_image = wa.unsqueeze(0)*Ia+wb.unsqueeze(0)*Ib+wc.unsqueeze(0)*Ic+wd.unsqueeze(0)*Id

  return perturbed_image.type(torch.float32).to(device)

In [ ]:
def default_flow_loss(flows,padding_mode='constant', epsilon=1e-8):
  paddings = (1,1,1,1)
  padded_flows = F.pad(flows,paddings,mode=padding_mode,value=0)
  shifted_flows = [
    padded_flows[:, :, 2:, 2:],  # bottom right (+1,+1)
    padded_flows[:, :, 2:, :-2],  # bottom left (+1,-1)
    padded_flows[:, :, :-2, 2:],  # top right (-1,+1)
    padded_flows[:, :, :-2, :-2]  # top left (-1,-1)
  ]
  #||\Delta u^{(p)} - \Delta u^{(q)}||_2^2 + # ||\Delta v^{(p)} - \Delta v^{(q)}||_2^2
  loss=0
  for shifted_flow in shifted_flows:

    #This should be the loss but works worse and is slower
    up_uq = torch.square(flows[:, 1] - shifted_flow[:, 1])
    vp_vq = torch.square(flows[:, 0] - shifted_flow[:, 0])
    print(up_uq.size())

    loss += torch.sum(torch.sqrt(up_uq + vp_vq + epsilon)).to(device)

  return loss.type(torch.float32)

def adv_loss(logits,targets,confidence=0.0):
  confidence=torch.tensor(confidence).to(device)
  real = torch.sum(logits*targets,-1)
  other = torch.max((1-targets)*logits-(targets*10000),-1)[0]
  return torch.max(other-real,confidence)[0].type(torch.float32)

In [ ]:
def modified_flow_loss(flows,padding_mode='constant', epsilon=1e-8):
  paddings = (1,1,1,1)
  padded_flows = F.pad(flows,paddings,mode=padding_mode,value=0)
  shifted_flows = [
    padded_flows[:, :, 2:, 2:],  # bottom right (+1,+1)
    padded_flows[:, :, 2:, :-2],  # bottom left (+1,-1)
    padded_flows[:, :, :-2, 2:],  # top right (-1,+1)
    padded_flows[:, :, :-2, :-2]  # top left (-1,-1)
  ]
  #||\Delta u^{(p)} - \Delta u^{(q)}||_2^2 + # ||\Delta v^{(p)} - \Delta v^{(q)}||_2^2
  loss=0
  for shifted_flow in shifted_flows:
    #Weird Loss:
    up_uq = torch.square(torch.norm(flows - shifted_flow, 2))
    vp_vq = torch.square(torch.norm(flows[:, 0] - shifted_flow[:, 0], 2))
#     print(up_uq.shape)

    #This should be the loss but works worse and is slower
    # up_uq = torch.square(flows[:, 1] - shifted_flow[:, 1])
    # vp_vq = torch.square(flows[:, 0] - shifted_flow[:, 0])

    loss += torch.sum(torch.sqrt(up_uq + vp_vq + epsilon)).to(device)

  return loss.type(torch.float32)

In [ ]:
losses = [default_flow_loss, modified_flow_loss]

In [ ]:
models[0].load_state_dict(torch.load("models/model.pt", map_location=device))
models[1].load_state_dict(torch.load("models/modelB.pt", map_location=device))
models[2].load_state_dict(torch.load("models/NetC.pt", map_location=device))

In [ ]:
def func(flows,input,target,model,flow_loss,const=0.05):
  input = torch.from_numpy(input).to(device)
  target = torch.from_numpy(target).to(device)
  flows = torch.from_numpy(flows).view((1,2,)+input.size()[2:]).to(device)
  flows.requires_grad=True
  pert_out = flow_st(input,flows)
  output = model(pert_out)
  L_flow = flow_loss(flows)
  L_adv = adv_loss(output,target)
  L_final = L_adv+const*L_flow
  model.zero_grad()
  L_final.backward()
  gradient = flows.grad.data.view(-1).detach().cpu().numpy()
  return L_final.item()

In [ ]:
def attack(input,target,model,flow_loss):
  init_flows = np.zeros((1,2,)+input.size()[2:]).reshape(-1)

  results = optimize.fmin_l_bfgs_b(func, init_flows, args=(input.cpu().numpy(), target.cpu().numpy(), model, flow_loss), approx_grad=True, epsilon=0.001)

  flows = torch.from_numpy(results[0]).view((1,2,) + input.size()[2:])
  pert_out = flow_st(input,flows)

  return pert_out

In [ ]:
class Attacker:
    def __init__(self, model, testset, loss_func, default_samples = None) -> None:
        self.model = model.to(device)
        # self.testset = testset
        self.loss_func = loss_func
        targets = testset.targets
        self.targeted_samples = dict()  # MNIST dataset split into groups by labels
        self.labels = torch.unique(targets, sorted=True).tolist()
        for label in self.labels:
            label_indices = testset.targets == label
            self.targeted_samples[label] = testset.data[label_indices]
        self.default_samples = self.get_full_sample() if default_samples is None else default_samples
    
    def set_model(self, new_model):
        self.model = new_model.to(device)
    
    def set_loss_func(self, new_loss_func):
        self.loss_func = new_loss_func

    def get_label_sample(self, label):
        num_samples = len(self.targeted_samples[label])
        source_img = (
            self.targeted_samples[label][random.randrange(0, num_samples)]
            .unsqueeze(0)
            .unsqueeze(0)
        )
        return source_img
    
    def get_full_sample(self):
        sample_shape = self.targeted_samples[self.labels[0]][0].size()
        sample_list = torch.zeros((len(self.labels), 1, 1) + sample_shape)
        for i, label in enumerate(self.labels):
            sample_list[i] = self.get_label_sample(label)
        return sample_list
    
    def reset_sample(self, label):
        self.default_samples[label] = self.get_label_sample(label)
    
    def reset_default_samples(self):
        self.default_samples = self.get_full_sample()

    def attack_from_img_to_target(self, source_img, target_label):
        target = torch.from_numpy(
            np.expand_dims(np.eye(10, dtype=np.float32)[target_label], 0)
        ).to(device)
        init_flows = np.zeros((1, 2) + source_img.size()[2:]).reshape(-1)

        results = optimize.fmin_l_bfgs_b(
            func,
            init_flows,
            args=(
                source_img.cpu().numpy(),
                target.cpu().numpy(),
                self.model,
                self.loss_func,
            ),
            approx_grad=True,
            epsilon=0.01
        )
        flows = torch.from_numpy(results[0]).view((1, 2) + source_img.size()[2:])
        pert_out = flow_st(source_img.to(device), flows.to(device))
        return pert_out

    def attack_from_to(self, source_label, target_label, new_sample=False):
        return self.attack_from_img_to_target(
            self.default_samples[source_label], target_label
        )

    def attack_from_img_to_all(self, source_img):
        sample_shape = self.targeted_samples[self.labels[0]][0].size()
        attack_list = torch.zeros((len(self.labels),) + sample_shape)
        for i, target in tqdm(enumerate(self.labels), total=len(self.labels), disable=True):
            attack_list[i] = self.attack_from_img_to_target(source_img, target)
        return attack_list

    def attack_all(self):
        sample_shape = self.targeted_samples[self.labels[0]][0].size()
        attack_grid = torch.zeros((len(self.labels), len(self.labels)) + sample_shape)
        for i, source_label in tqdm(enumerate(self.labels), total=len(self.labels), disable=False):
            source_img = self.default_samples[i]
            attack_grid[i] = self.attack_from_img_to_all(source_img)
        return attack_grid

    def print_attack_list(self, attack_list, color_failed=False):
        fig = plt.figure(
            figsize=(len(self.labels), 1),
        )
        fig.subplots_adjust(wspace=0, hspace=0)
        ax = [
            fig.add_subplot(1, len(self.labels), i + 1) for i in range(len(self.labels))
        ]
        for i, target_label in enumerate(self.labels):
            ax[i].set_xticks([], [])
            ax[i].set_yticks([], [])
            im = attack_list[i]
            if color_failed and (
                torch.argmax(
                    self.model(attack_list[i].unsqueeze(0).unsqueeze(0).to(device))
                ).item()
                != target_label
            ):
                im = torch.zeros(attack_list[i].shape + (3,))
                im[:, :, 0] = attack_list[i]
            ax[i].imshow(im, cmap="gray")
            # ax[i].set_aspect("equal")
            ax[i].set_title(f"{i}")

        plt.savefig(f"figures/one_to_all.pdf")

    def print_attack_grid(self, attack_grid, color_failed=False):
        fig = plt.figure(
            figsize=(len(self.labels), len(self.labels)),
        )
        fig.subplots_adjust(wspace=0, hspace=0)
        ax = [
            fig.add_subplot(len(self.labels), len(self.labels), i + 1)
            for i in range(len(self.labels) ** 2)
        ]
        cnt = 0
        for i, source_label in enumerate(self.labels):
            for j, target_label in enumerate(self.labels):
                ax[cnt].set_xticks([], [])
                ax[cnt].set_yticks([], [])
                im = attack_grid[i, j]
                output = self.model(attack_grid[i, j].unsqueeze(0).unsqueeze(0).to(device))
                pred = torch.argmax(output).item()
                if color_failed and i != j and pred == source_label:
                    im = torch.zeros(attack_grid[i, j].shape + (3,))
                    im[:, :, 0] = (attack_grid[i, j] - attack_grid[i, j].min()) / attack_grid[i,j].max()
                ax[cnt].imshow(im, cmap="gray")
                ax[cnt].set_aspect("equal")
                if i == 0:
                    ax[cnt].set_title(f"{j}", fontsize=25)
                cnt += 1

        plt.savefig(f"figures/all_to_all_{self.loss_func.__name__}_{type(self.model).__name__}_1.jpg", bbox_inches='tight', pad_inches=0)

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
testset = datasets.MNIST(root = 'data', train=False, transform = transform, download=True)

In [ ]:
attacker = Attacker(models[0], testset, default_flow_loss)

In [ ]:
for model in models:
    for flow_loss in losses:
        attacker.set_model(model)
        attacker.set_loss_func(flow_loss)
        attack_grid = attacker.attack_all()
        attacker.print_attack_grid(attack_grid)

In [ ]:
default_a = np.asarray(Image.open("figures/all_to_all_default_flow_loss_NetA.jpg"))
default_b = np.asarray(Image.open("figures/all_to_all_default_flow_loss_NetB.jpg"))
default_c = np.asarray(Image.open("figures/all_to_all_default_flow_loss_NetC.jpg"))
weird_a = np.asarray(Image.open("figures/all_to_all_weird_flow_loss_NetA.jpg"))
weird_b = np.asarray(Image.open("figures/all_to_all_weird_flow_loss_NetB.jpg"))
weird_c = np.asarray(Image.open("figures/all_to_all_weird_flow_loss_NetC.jpg"))

In [ ]:
defaults = [default_a, default_b, default_c]
weirds = [weird_a, weird_b, weird_c]
names = ["Net A", "Net B", "Net C"]

In [ ]:
fig = plt.figure(figsize=(12, 4))
fig.subplots_adjust(wspace=0, hspace=0)
ax = [
    fig.add_subplot(1, 3, i + 1) for i in range(3)
]
for i in range(3):
    ax[i].set_xticks([], [])
    ax[i].set_yticks([], [])
    im = weirds[i]
    ax[i].imshow(im, cmap="gray")
    ax[i].set_aspect("equal")
    ax[i].spines['top'].set_visible(False)
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['bottom'].set_visible(False)
    ax[i].spines['left'].set_visible(False)
    ax[i].set_title(names[i], fontsize=8)
fig.suptitle("Paper implementation flow loss")

plt.savefig(f"figures/fig_2_modified_loss.jpg", bbox_inches="tight")

In [ ]:
def get_indexes(correct, adv):
    indices = []
    for i in range(10):
      for j in range(10):
        indices  += [index for (index, item) in enumerate(correct_label) if (item == i and adv[index] == j)]
    return indices

In [ ]:
numbers = []
for i in range(10):
    l = [num for num in range(10)]
    numbers.append(l)
def give_label(label, d):
    if  len(d[label]) == 0:
        number = np.random.choice([num for num in range(10) if num != label])
    else:
        number = d[label].pop()
    return number

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
testset = datasets.MNIST(root = './data', train=False, transform = transform, download=True)
test_loader = torch.utils.data.DataLoader(testset,batch_size=1,shuffle=True)

# use_cuda=True
# device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

for model in models[0:1]:
  for flow_loss in losses[1:2]:
    # model = model.to(device)
    # model.load_state_dict(torch.load(f'./{type(model).__name__}.pt'))
    model.eval()

    adv=[]
    adv_label=[]
    correct_label=[]
    failed_inputs=[]
    failed_targets=[]
    sample=100
    success_untargeted=0
    success_targeted=0
    print(f"Model: {type(model).__name__}")
    print(f"Loss: {flow_loss.__name__}")
    for i,data in tqdm(enumerate(test_loader), total=sample):
      if i<sample:
        input,label = data[0].to(device),data[1].to(device)
        # break
        random_target = np.random.choice([num for num in range(10) if num != label.item()])
        target = torch.from_numpy(np.expand_dims(np.eye(10, dtype=np.float32)[random_target],0)).to(device)
        pert_out = attack(input,target,model.to(device), flow_loss)
        if pert_out is None:
          failed_inputs.append(input)
          failed_targets.append(random_target)
        if pert_out!=None:
          output =  model(pert_out)
          adv.append(pert_out.squeeze().detach().cpu().numpy())
          correct_label.append(label.item())
          adv_label.append(torch.argmax(output).item())
          if label.item()!=torch.argmax(output).item():
            success_untargeted +=1
          if random_target == torch.argmax(output).item():
            success_targeted += 1
      else:
        break
    print("Success Rate (Untargeted): {:.4f} ".format(success_untargeted/sample))
    print("Success Rate (Targeted): {:.4f} ".format(success_untargeted/sample))
    cnt = 0
    plt.figure(figsize=(10,10))

    for i in range(len(adv)):
      cnt += 1
      plt.subplot(10,10,cnt)
      plt.xticks([], [])
      plt.yticks([], [])
      orig,adv_,ex = correct_label[i],adv_label[i],adv[i]
      plt.title("{} -> {}".format(orig, adv_))
      plt.imshow(ex, cmap="gray")

    plt.tight_layout()

    plt.savefig(f"figures/images_{flow_loss.__name__}_{type(model).__name__}.pdf")